# Allegro Web API

In [1]:
import hashlib
import base64
import time
import numpy as np
import pandas as pd
import pickle

from suds.client import Client, WebFault

In [2]:
api_key = 'xxx'
api_key_sandbox = 'xxx'
login = 'xxx'
password = 'xxx'

In [3]:
# class AllegroWebAPI:
#     API_URL_sandbox = 'https://webapi.allegro.pl.webapisandbox.pl/service.php?wsdl'
#     API_URL = 'https://webapi.allegro.pl/service.php?wsdl'
#     country_code = 1 # 1 dla Polski, 56 dla Czech, 209 dla Ukrainy
 
#     def __init__(self, login, password, api_key):
#         self.login = login
#         self.password_enc = base64.b64encode(hashlib.sha256(password.encode('utf-8')).digest()).decode()
#         self.api_key = api_key
#         self.client = Client(self.API_URL)
#         self.service = self.client.service
#         self.version_key = self.get_version_by_country_code()
#         self.item_id_list = []
#         self.item_dict = {}
 
#     def get_version_by_country_code(self):
#         systems = self.service.doQueryAllSysStatus(**{
#             'countryId': self.country_code,
#             'webapiKey':self.api_key
#         })[0]
 
#         for sys in systems:
#             if sys['countryId'] == self.country_code:
#                 return sys['verKey']
 
#     def sign_in_enc(self):
#         try:
#             self.session_id = self.service.doLoginEnc(
#                 userLogin= self.login,
#                 userHashPassword = self.password_enc,
#                 countryCode = self.country_code,
#                 webapiKey = self.api_key,
#                 localVersion = self.version_key
#             )
#         except WebFault as e:
#             print e
            
#     def get_categories_tree(self):
#         self.categories_tree = self.service.doGetCatsData(self.country_code, self.version_key, self.api_key)
    
#     def get_books(self, filter_value_min=30, filter_value_max=31, result_size=200):
#         self.item_last_cost = filter_value_min
        
#         for cost in np.arange(filter_value_min, filter_value_max, 0.10):
            
#             cost2 = cost + 0.099
#             if cost2 > filter_value_max:
#                 cost2 = filter_value_max
#             self.item_last_cost = cost + 0.10
            
#             table = {'price': {'filterValueRange': [cost, cost2]},
#                      'category': {'filterValueId': '79156'}
#                     }
#             filtr_query = api.client.factory.create('ArrayOfFilteroptionstype')

#             for i in range(len(table)):
#                 filtr = api.client.factory.create('FilterOptionsType')
#                 filtr.filterId = table.keys()[i]
#                 if 'filterValueId' in table.values()[i].keys():
#                     filtrAOS = api.client.factory.create('ArrayOfString')
#                     filtrAOS.item = table.values()[i].values()[0]
#                     filtr.filterValueId = filtrAOS
#                 if 'filterValueRange' in table.values()[i].keys():
#                     filtrRVT = api.client.factory.create('RangeValueType')
#                     filtrRVT.rangeValueMin = table.values()[i].values()[0][0]
#                     filtrRVT.rangeValueMax = table.values()[i].values()[0][1]
#                     filtr.filterValueRange = filtrRVT
#                 filtr_query.item.append(filtr)

#             item_list = api.service.doGetItemsList(api.api_key, api.country_code, filtr_query, resultSize = result_size)
            
#             print time.strftime("%A, %H:%M:%S"), cost, cost + 0.099

#             for item in item_list.itemsList.item:
#                 item_id = item.itemId
#                 item_title = item.itemTitle
#                 if item_id not in self.item_dict.keys():
#                     self.item_id_list.append(item_id)
#                     self.item_dict[str(item_id)] = {'title': item_title}
# #                     full_item = api.service.doShowItemInfoExt(api.session_id.sessionHandlePart, item_id, 
# #                                                               getDesc = 0).itemListInfoExt
# #                     self.item_dict[str(item_id)] = {'name': full_item.itName, 'description': full_item.itDescription}
    
#     def get_books_until_done(self, filter_value_min=30, filter_value_max=31, result_size=200):
#         try:
#             api.get_books(filter_value_min, filter_value_max, result_size)
#         except WebFault as e:
#             print 'Probujemy po bledzie'
#             api.sign_in_enc()
#             api.get_books_until_done(api.item_last_cost, filter_value_max, result_size)
            
#     def transform_dict_to_DataFrame(self):
#         df = pd.DataFrame(self.item_dict)
#         df = df.transpose()
#         self.item_DataFrame = df

In [4]:
class AllegroWebAPI:
    API_URL_sandbox = 'https://webapi.allegro.pl.webapisandbox.pl/service.php?wsdl'
    API_URL = 'https://webapi.allegro.pl/service.php?wsdl'
    country_code = 1 # 1 dla Polski, 56 dla Czech, 209 dla Ukrainy
 
    def __init__(self, login, password, api_key):
        self.login = login
        self.password_enc = base64.b64encode(hashlib.sha256(password.encode('utf-8')).digest()).decode()
        self.api_key = api_key
        self.client = Client(self.API_URL)
        self.service = self.client.service
        self.version_key = self.get_version_by_country_code()
        self.item_id_list = []
        self.item_dict = {}
        self.item_offset = 0
 
    def get_version_by_country_code(self):
        systems = self.service.doQueryAllSysStatus(**{
            'countryId': self.country_code,
            'webapiKey':self.api_key
        })[0]
 
        for sys in systems:
            if sys['countryId'] == self.country_code:
                return sys['verKey']
 
    def sign_in_enc(self):
        try:
            self.session_id = self.service.doLoginEnc(
                userLogin= self.login,
                userHashPassword = self.password_enc,
                countryCode = self.country_code,
                webapiKey = self.api_key,
                localVersion = self.version_key
            )
        except WebFault as e:
            print e
            
    def get_categories_tree(self):
        self.categories_tree = self.service.doGetCatsData(self.country_code, self.version_key, self.api_key)
    
    def get_books(self, filter_value_min=30, filter_value_max=31, result_size=200):
        
        table = {'price': {'filterValueRange': [filter_value_min, filter_value_max]},
                 'category': {'filterValueId': '79156'}
                }
        filtr_query = api.client.factory.create('ArrayOfFilteroptionstype')
        for i in range(len(table)):
            filtr = api.client.factory.create('FilterOptionsType')
            filtr.filterId = table.keys()[i]
            if 'filterValueId' in table.values()[i].keys():
                filtrAOS = api.client.factory.create('ArrayOfString')
                filtrAOS.item = table.values()[i].values()[0]
                filtr.filterValueId = filtrAOS
            if 'filterValueRange' in table.values()[i].keys():
                filtrRVT = api.client.factory.create('RangeValueType')
                filtrRVT.rangeValueMin = table.values()[i].values()[0][0]
                filtrRVT.rangeValueMax = table.values()[i].values()[0][1]
                filtr.filterValueRange = filtrRVT
            filtr_query.item.append(filtr)
            
        item_list = self.service.doGetItemsList(self.api_key, self.country_code, filtr_query, resultSize = result_size)
        
        while hasattr(item_list, "itemsList"):
            
            print time.strftime("%A, %H:%M:%S"), "Offset: ", self.item_offset

            for item in item_list.itemsList.item:
                item_id = item.itemId
                item_title = item.itemTitle
                if item_id not in self.item_dict.keys():
                    self.item_id_list.append(item_id)
                    self.item_dict[str(item_id)] = {'title': item_title}
            
            self.item_offset = len(self.item_id_list)
            item_list = self.service.doGetItemsList(self.api_key, self.country_code, filtr_query, 
                                                   resultSize = result_size, resultOffset = self.item_offset)
    
    def get_books_until_done(self, filter_value_min=30, filter_value_max=31, result_size=200):
        try:
            api.get_books(filter_value_min, filter_value_max, result_size)
        except WebFault as e:
            print 'Probujemy po bledzie'
            api.sign_in_enc()
            api.get_books_until_done(api.item_last_cost, filter_value_max, result_size)
    
    def get_top_n_books_query(self, query, filter_value_min=30, filter_value_max=31, top_n=200):
        
        self.item_top_n_id_list = []
        self.item_top_n_dict = {}        
        
        table = {'price': {'filterValueRange': [filter_value_min, filter_value_max]},
                 'category': {'filterValueId': '79156'},
                 'search': {'filterValueId': query}
                }
        
        filtr_query = api.client.factory.create('ArrayOfFilteroptionstype')
        for i in range(len(table)):
            filtr = api.client.factory.create('FilterOptionsType')
            filtr.filterId = table.keys()[i]
            if 'filterValueId' in table.values()[i].keys():
                filtrAOS = api.client.factory.create('ArrayOfString')
                filtrAOS.item = table.values()[i].values()[0]
                filtr.filterValueId = filtrAOS
            if 'filterValueRange' in table.values()[i].keys():
                filtrRVT = api.client.factory.create('RangeValueType')
                filtrRVT.rangeValueMin = table.values()[i].values()[0][0]
                filtrRVT.rangeValueMax = table.values()[i].values()[0][1]
                filtr.filterValueRange = filtrRVT
            filtr_query.item.append(filtr)
            
        sort_query = api.client.factory.create('SortOptionsType')
        sort_query.sortType = 'relevance'
        sort_query.sortOrder = 'desc'
        
        item_list = self.service.doGetItemsList(self.api_key, self.country_code, filtr_query, sort_query, resultSize = top_n)
        
        for item in item_list.itemsList.item:
            item_id = item.itemId
            item_title = item.itemTitle
            if item_id not in self.item_dict.keys():
                self.item_top_n_id_list.append(item_id)
                self.item_top_n_dict[str(item_id)] = {'title': item_title}
    
    def transform_dict_to_DataFrame(self, top_n = False):
        if top_n:
            df = pd.DataFrame(self.item_top_n_dict)
            df = df.transpose()
            self.item_top_n_DataFrame = df
        else:
            df = pd.DataFrame(self.item_dict)
            df = df.transpose()
            self.item_DataFrame = df

In [5]:
api = AllegroWebAPI(login, password, api_key)
api.sign_in_enc()

In [6]:
api.get_books_until_done(0, 5000, 1000)

Friday, 21:42:38 Offset:  0
Friday, 21:43:05 Offset:  1000
Friday, 21:43:19 Offset:  2000
Friday, 21:43:39 Offset:  3000
Friday, 21:44:03 Offset:  4000
Friday, 21:44:24 Offset:  5000
Friday, 21:44:41 Offset:  6000
Friday, 21:45:04 Offset:  7000
Friday, 21:45:18 Offset:  8000
Friday, 21:45:34 Offset:  9000
Friday, 21:45:56 Offset:  10000
Friday, 21:46:18 Offset:  11000
Friday, 21:46:42 Offset:  12000
Friday, 21:46:56 Offset:  13000
Friday, 21:47:11 Offset:  14000
Friday, 21:47:31 Offset:  15000
Friday, 21:47:48 Offset:  16000
Friday, 21:48:02 Offset:  17000
Friday, 21:48:19 Offset:  18000
Friday, 21:48:34 Offset:  19000
Friday, 21:48:56 Offset:  20000
Friday, 21:49:18 Offset:  21000
Friday, 21:49:33 Offset:  22000
Friday, 21:49:47 Offset:  23000
Friday, 21:50:10 Offset:  24000
Friday, 21:50:27 Offset:  25000
Friday, 21:50:42 Offset:  26000
Friday, 21:50:59 Offset:  27000
Friday, 21:51:15 Offset:  28000
Friday, 21:51:31 Offset:  29000
Friday, 21:51:49 Offset:  30000
Friday, 21:52:20 Offs

In [7]:
api.transform_dict_to_DataFrame()

In [8]:
api.item_DataFrame.to_csv('titles_books.csv', encoding='utf-8')

In [9]:
with open('titles_books_DataFrame.pickle', 'wb') as handle:
    b = pickle.dump(api.item_DataFrame, handle)

In [10]:
with open('titles_books_dict.pickle', 'wb') as handle:
    b = pickle.dump(api.item_dict, handle)

# Dodatkowy kod dla drzewa kategorii

In [ ]:
api.categories_tree

In [ ]:
category_name = 'Książki i Komiksy'
api.get_categories_tree()
for category in api.categories_tree.catsList.item:
    if category['catName'] == category_name:
        category_id = category['catId']
        break
    

In [ ]:
api.service.doGetCategoryPath(api.session_id.sessionHandlePart, 91459)